In [193]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import re
import nltk
from gensim import corpora, models
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np

In [194]:
embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')


In [195]:
#Emoji Removal

def deEmojify(x):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'', x)

In [196]:
#Cleaning basic Characters

def clean(raw):
    """ Remove hyperlinks and markup """
    result = re.sub("<[a][^>]*>(.+?)</[a]>", 'Link.', raw)
    result = re.sub('&gt;', "", result)
    result = re.sub('&#x27;', "'", result)
    result = re.sub('&quot;', '"', result)
    result = re.sub('&#x2F;', ' ', result)
    result = re.sub('<p>', ' ', result)
    result = re.sub('</i>', '', result)
    result = re.sub('&#62;', '', result)
    result = re.sub('<i>', ' ', result)
    result = re.sub("\n", '', result)
    return result

In [197]:
def remove_num(texts):
    output = re.sub(r'\d+', '', texts )
    return output

In [198]:
def unify_whitespaces(text):
    cleaned_string = re.sub(' +', ' ', text )
    return cleaned_string

In [199]:
def remove_punctuation(text):
    result = "".join(u for u in text if u not in ("?", ".", ";", ":",  "!",'"',',') )
    return result

In [200]:
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
nltk.download('stopwords')
from nltk.stem import PorterStemmer

stop = set(stopwords.words('english')).union({'unsubscribe', 'min', 'answer', 'question', 'please', 'email', 'web', 'see', 'link', 'click', 'like', 'new', 'one', 'take', 'look', 'recent', 'message'})
#add unsubscribe to stop
stop.add('unsubscribe')
stemmer = PorterStemmer()
lemma = WordNetLemmatizer()

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop ]
    return ' '.join(text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aayush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [201]:
def preprocess(line):
    line=re.sub(r'http\S+', '', line.strip())
    line = re.sub(r'[!"#$%&\'()*+,-./:;?@[\\\]^_{|}~`]', '', line)
    line=deEmojify(line)
    line=clean(line)
    line=remove_num(line)
    line=unify_whitespaces(line)
    line=remove_punctuation(line)
    line=remove_stopwords(line)
    return line


In [202]:
file_path = './emails.csv'
lines = []
comments = []
with open(file_path, 'r', encoding = 'utf-8') as file:
    for line in file:
        if line.strip()!='""':
            comments.append(line)
            line = preprocess(line)
            lines.append(line)
lines=lines[1:]
comments = comments[1:]
print(len(lines))
print(lines[1])

780
dear innovator congratulations delighted inform cohort idea successfully reserved place prototyping stage result achievement fills us great satisfaction wholeheartedly celebrate progress firmly believe innovative idea potential bring meaningful change society committed supporting journey next step exciting process curated series captivating opportunities designed ensure smooth productive start prototyping phase cordially invite orientation session provide comprehensive insights prototyping stage well overview forthcoming events mentoring programs resources intended guide towards structuring advancing initiative session details date september time pm pm platform webex confirm attendance orientation session kindly button register registration following orientation session keep informed additional activities aimed enhancing experience prototyping stage including oneonone mentorship access resources prototype development engaging events encourage bring questions ideas orientation sessi

In [203]:
corpus_embeddings = embedder.encode(lines)

In [204]:
corpus_embeddings[1]

array([ 3.07398677e-01,  8.72956038e-01,  5.16074419e-01, -8.00583303e-01,
        2.01442629e-01, -3.00424322e-02, -3.76271635e-01, -1.37540370e-01,
        2.24327132e-01, -7.87486851e-01, -2.93627799e-01,  6.30936623e-01,
        2.58255720e-01,  3.12938720e-01,  4.51004803e-01, -5.11695445e-01,
        7.86232591e-01, -8.64083469e-02, -5.19522190e-01, -2.01813038e-02,
        4.76739019e-01,  2.23206401e-01, -2.34065101e-01, -2.05037087e-01,
        2.34904304e-01, -1.24744639e-01, -1.39900800e-02,  4.95046586e-01,
        6.86075568e-01,  6.17228687e-01,  2.88830996e-01, -2.52360642e-01,
       -1.17199682e-01, -1.54682398e-01,  7.97579661e-02,  5.43694377e-01,
       -7.32851088e-01, -1.82486057e-01, -6.63464904e-01, -1.25397488e-01,
       -1.50133252e-01, -2.11476311e-01,  7.91223288e-01, -1.60256997e-01,
       -1.66030630e-01,  5.26648581e-01, -5.87493956e-01,  3.08176637e-01,
       -7.13940620e-01,  2.73188382e-01, -2.93809891e-01, -1.07242596e+00,
       -8.38404357e-01, -

In [205]:
# Then, we perform k-means clustering using sklearn:
from sklearn.cluster import KMeans

num_clusters = 5
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

C:\Users\Aayush\AppData\Roaming\Python\Python310\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [206]:
comment = 'answer'
preprocessed = preprocess(comment)
test=embedder.encode([preprocessed])
clustering_model.predict(test)[0]

3

In [207]:
cluster_assignment

print(len(cluster_assignment))
print(len(comments))

comment_cluster = zip(comments, cluster_assignment)

result = list(comment_cluster)
print(result[0])
clustered_comments = {}
for comment, cluster in result:
    if cluster not in clustered_comments:
        clustered_comments[cluster] = []
    clustered_comments[cluster].append(comment)

# Convert the dictionary to a 2D array
clustered_comments_array = [clustered_comments.get(i, []) for i in range(0, max(clustered_comments.keys()) + 1)]

# Print the result
for cluster, comments in enumerate(clustered_comments_array, start=0):
    print(f"Cluster {cluster}: {comments}")

780
780
('"Hello Innovator, What if we told you that you could work with your dream company straight out of college? Yes! You heard it right! Participate in the Accenture Innovation Challenge to win big and you may also get an opportunity to fast-track your recruitment process journey with Accenture. Application Link: https://unstop.com/o/l58dzh1/?ref=h1FyMTt0 Roles Offered: Associate Software Engineer System and Application Services Associate Rewards: ❖ Prizes worth INR 1,00,000 per team member❖ Fast-Track Interview Opportunity Eligibility: Undergraduate and Postgraduate students Participate in the most sought-after challenge, and let your ideas make waves. Note that the fast-track recruitment process will be offered to all the eligible (as per our recruitment process criteria) participants who would submit their innovative ideas. Eligible participants will receive communications about the recruitment process from Accenture. Best Regards,Team Unstop Unsubscribe here"\n', 1)
Cluster 0:

Cluster 1 Topics:
[(0, '0.043*"https" + 0.015*"read" + 0.012*"3d" + 0.007*"question" + 0.007*"min" + 0.006*"answer" + 0.005*"new" + 0.004*"like" + 0.003*"ai" + 0.003*"1"'), (1, '0.000*"https" + 0.000*"read" + 0.000*"3d" + 0.000*"min" + 0.000*"question" + 0.000*"answer" + 0.000*"new" + 0.000*"like" + 0.000*"1" + 0.000*"ai"'), (2, '0.001*"https" + 0.000*"read" + 0.000*"3d" + 0.000*"question" + 0.000*"min" + 0.000*"answer" + 0.000*"new" + 0.000*"like" + 0.000*"data" + 0.000*"ai"'), (3, '0.000*"https" + 0.000*"read" + 0.000*"3d" + 0.000*"min" + 0.000*"question" + 0.000*"answer" + 0.000*"new" + 0.000*"like" + 0.000*"rs" + 0.000*"one"'), (4, '0.000*"https" + 0.000*"read" + 0.000*"3d" + 0.000*"question" + 0.000*"min" + 0.000*"1" + 0.000*"answer" + 0.000*"like" + 0.000*"rs" + 0.000*"ai"')]


Cluster 2 Topics:
[(0, '0.000*"https" + 0.000*"read" + 0.000*"2023" + 0.000*"3d" + 0.000*"students" + 0.000*"ieee" + 0.000*"unsubscribe" + 0.000*"inr" + 0.000*"us" + 0.000*"internship"'), (1, '0.000*"https

In [ ]:
clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(lines[sentence_id])



In [ ]:
print(clustered_sentences[0][0])

weeks theme words ai usage examples secundan sekuhnduhn adjective occurring every day latin secundus second earliest documented use c water conservation ad brisbane australia photo tamara forsooth sir henry hath adopted curious habit taking long walks secundan morrows sun offends next shines william shakespeare tales times oddities stratford sentinel england jun aigenerated usage example todays sponsor orijinz fabulous game laughed laughed hours play family friends smash much fun wealth passed merit bad luck seen bad character ideologues justify punishing sick poor poverty neither crime character flaw stigmatize let people die struggle live sarah kendzior journalist author b sep looking wordquotation archives change address etc pronunciation permalink


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert the clustered sentences into separate documents for TF-IDF analysis
clustered_documents = [' '.join(cluster) for cluster in clustered_sentences]

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the clustered documents
tfidf_matrix = tfidf_vectorizer.fit_transform(clustered_documents)

# Get the feature names (words) from the TF-IDF vectorizer
feature_names = tfidf_vectorizer.get_feature_names_out()

# For each cluster, find the top N keywords
num_keywords = 5  # You can adjust this value as needed
cluster_labels = []

for i, cluster_indices in enumerate(tfidf_matrix):
    # Get the indices of the top N keywords
    feature_index = cluster_indices.toarray().argsort()[0, -num_keywords:]
    cluster_keywords = [feature_names[idx] for idx in feature_index]
    cluster_labels.append(cluster_keywords)

# Print the cluster labels
for i, labels in enumerate(cluster_labels):
    print(f"Cluster {i} - Labels: {', '.join(labels)}")


Cluster 0 - Labels: time, data, rs, ai, read
Cluster 1 - Labels: read, internship, ieee, inr, students
Cluster 2 - Labels: group, university, campus, linkedin, pesu
Cluster 3 - Labels: florida, florence, flower, カラン, version
Cluster 4 - Labels: exchange, और, आपक, broker, कर


In [ ]:
for i in cluster_labels:
    print(i)

['time', 'data', 'rs', 'ai', 'read']
['read', 'internship', 'ieee', 'inr', 'students']
['group', 'university', 'campus', 'linkedin', 'pesu']
['florida', 'florence', 'flower', 'カラン', 'version']
['exchange', 'और', 'आपक', 'broker', 'कर']
